<a href="https://colab.research.google.com/github/bcataldo3/plantilla_curso_python/blob/main/docs/Clase_05 - Modelo K-Medias Metricas Evaluacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACHINE LEARNING NO SUPERVISADO
## Modelo K-medias - Métricas de Evaluación

### Minimizar la Inercia

Uno de los mayores inconvenientes de K-medias y algoritmos similares es la solicitud explícita del número de agrupaciones. A veces, esta información está impuesta por restricciones externas (por ejemplo, en el caso de estudio, solo hay dos posibles diagnósticos), pero en muchos casos (cuando se necesita un análisis exploratorio), el científico de datos debe verificar diferentes configuraciones y evaluarlas. La forma más sencilla de evaluar el rendimiento de K-medias y elegir un número apropiado de grupos se basa en la comparación de diferentes inercias finales. Para explicar esto mediremos las inercias de un ejemplo generado a partir de 12 globos Gausianos muy compactos generados con la funcion de ```scikit-learn make_blobs()```

Comenzamos creando la data y mostrando resultados

<img src="https://drive.google.com/uc?export=view&id=1vuLtM8DQhLmKcN6C3-j9DknugLiwDwqt" width = "800" align="center"/>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
X, Y = make_blobs(n_samples=2000, n_features=2, centers=12,cluster_std=0.05, center_box=[-5, 5], random_state=100)
# Show the blobs
sns.set()
fig, ax = plt.subplots(figsize=(12, 8))
for i in range(12):
    ax.scatter(X[Y == i, 0], X[Y == i, 1], label='Blob {}'.format(i + 1))
ax.set_xlabel(r'$x_0$')
ax.set_ylabel(r'$x_1$')
ax.legend()
plt.show()

Ahora mostramos la grafica de la inercia para distintos números de agrupaciones y el código correspondiente.

<img src="https://drive.google.com/uc?export=view&id=1_HtS5eeEoJd6jhVeScIHm-fHtXMcI_tk" width = "800" align="center"/>

In [ ]:
# Compute the inertia
inertias = []
for i in range(2, 21):
    km = KMeans(n_clusters=i, max_iter=1000, random_state=1000)
    km.fit(X)
    inertias.append(km.inertia_)
# Show the plot inertia vs. no. clusters
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(np.arange(2, 21, 1), inertias)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Inertia', fontsize=14)
ax.set_xticks(np.arange(2, 21, 1))
plt.show()

Algunos comentarios del proceso:
* El gráfico anterior muestra un comportamiento común. Cuando el número de grupos es muy pequeño, la densidad es proporcionalmente baja, por lo tanto, la cohesión es baja y, como resultado, la inercia es alta. Aumentar el número de grupos obliga al modelo a crear grupos más cohesionados y la inercia comienza a disminuir abruptamente. 
* La regla heurística genérica (cuando no hay restricciones externas) es elegir el número de grupos correspondiente al punto que separa la región de alta variación de la casi plana. De esta manera, nos aseguramos de que todos los grupos hayan alcanzado su máxima cohesión sin fragmentación interna. 
* Por supuesto, en este caso, si hubiéramos seleccionado K=15, nueve conjuntos habrían sido asignados a diferentes grupos, mientras que los otros tres se habrían dividido en dos partes. Obviamente, al dividir una región de alta densidad, la inercia sigue siendo baja, pero el principio de separación máxima ya no se sigue.

### Caso de Estudio: Minimizando la Inercia
Ahora revisaremos la inercia de nuestro caso de estudio para diferentes niveles de conglomerados (de 2 a 50)

<img src="https://drive.google.com/uc?export=view&id=1eV3JwsWjejcRNbhmN4YyusC6wNbfPL-z" width = "800" align="center"/>


In [ ]:
import pandas as pd
url='https://drive.google.com/file/d/1LG6hb0duLtnSiFhT9JWFNuo8qDxHV08W/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

dataset_columns = ['id','diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean','area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean','concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','radius_se','texture_se', 'perimeter_se', 'area_se', 'smoothness_se','compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se','fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst','area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst','concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df = pd.read_csv(url, sep="," ,index_col=0, names=dataset_columns)
cdf = df.drop(['diagnosis'], axis=1)

# Compute the inertia
inertias = []

for i in range(2, 51):
    km = KMeans(n_clusters=i, max_iter=1000, random_state=1000)
    km.fit(cdf)
    inertias.append(km.inertia_)

# Show the plot inertia vs. no. clusters
fig, ax = plt.subplots(figsize=(18, 8))

ax.plot(np.arange(2, 51, 1), inertias)
ax.set_xlabel('Number of clusters', fontsize=14)
ax.set_ylabel('Inertia', fontsize=14)
ax.set_xticks(np.arange(2, 51, 2))

plt.show()

En este caso, la verdad fundamental sugiere que deberíamos agrupar en dos grupos correspondientes a los diagnósticos. Sin embargo, el gráfico muestra un descenso drástico que termina en K=8 y continúa con una pendiente más baja hasta aproximadamente K=40. Durante el análisis preliminar, hemos observado que la proyección bidimensional está compuesta por muchos conjuntos aislados que comparten el mismo diagnóstico. Por lo tanto, podríamos decidir emplear, por ejemplo, K=8 y analizar las características correspondientes a cada grupo. Dado que esto no es una tarea de clasificación, la verdad fundamental puede usarse como la principal referencia, pero un análisis exploratorio correcto puede intentar comprender la composición de las subestructuras para proporcionar detalles adicionales a los técnicos (por ejemplo, médicos). Ahora realizaremos un agrupamiento K-medias con ocho grupos en el conjunto de datos para describir la estructura de dos grupos de muestra, de la siguiente manera:

In [ ]:
# Preparamos nuevamente la data del metodo t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=10, random_state=1000)
data_tsne = tsne.fit_transform(cdf)
df_tsne = pd.DataFrame(data_tsne, columns=['x', 'y'], index=cdf.index)
dff = pd.concat([df, df_tsne], axis=1)

km = KMeans(n_clusters=8, max_iter=1000, random_state=1000)
Y_pred = km.fit_predict(cdf)
df_km = pd.DataFrame(Y_pred, columns=['prediction'], index=cdf.index)
kmdff = pd.concat([dff, df_km], axis=1)

# Show the result
fig, ax = plt.subplots(figsize=(18, 11))
with sns.plotting_context("notebook", font_scale=1.5):
    sns.scatterplot(x='x',y='y',hue='prediction',size='area_mean',style='diagnosis',sizes=(30, 400),palette=sns.color_palette("husl", 8),data=kmdff,ax=ax)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
plt.show()

<img src="https://drive.google.com/uc?export=view&id=1QO9vXXRJb2ou4V_Fg6wDa_vAcu3m1cc1" width = "800" align="center"/>

Ahora, consideremos el intervalo $-25 < x < 30$ y $-60 < y <-40$ y revisemos estadística descriptiva sencilla de algunos de sus atributos.

In [ ]:
sdff_1 = dff[(dff.x > -25.0) & (dff.x < 30.0) & (dff.y > -60.0) & (dff.y < -40.0)]
print(sdff_1[['perimeter_mean', 'area_mean', 'smoothness_mean','concavity_mean', 'symmetry_mean']].describe())

In [ ]:
       perimeter_mean    area_mean  ...  concavity_mean  symmetry_mean
count       52.000000    52.000000  ...       52.000000      52.000000
mean       129.767308  1198.030769  ...        0.178122       0.195225
std          6.442467    99.306666  ...        0.064752       0.026669
min        110.000000   904.600000  ...        0.086900       0.144800
25%        125.325000  1136.000000  ...        0.136800       0.179950
50%        130.200000  1206.500000  ...        0.161000       0.191200
75%        133.950000  1277.500000  ...        0.212725       0.209425
max        143.700000  1386.000000  ...        0.375400       0.290600

Desde la verdad fundamental, sabemos que todas estas muestras son malignas, pero podemos intentar determinar una regla. La relación entre el área media y el perímetro medio es aproximadamente de 9.23 y las desviaciones estándar relativas son muy pequeñas en comparación con las medias. Esto significa que estas muestras representan tumores extendidos en un rango muy estrecho. Además, tanto la concavidad media como la simetría media son mayores que los valores generales. Por lo tanto (sin la presunción de un análisis científicamente razonable), podemos concluir que las muestras asignadas a estos grupos representan tumores muy graves que han alcanzado una etapa avanzada. Ahora revisemos la siguiente área $x> -10$ y $ 20 < y
< 50$

In [ ]:
sdff_2 = dff[(dff.x > -10.0) & (dff.y > 20.0) & (dff.y < 50.0)]
print(sdff_2[['perimeter_mean', 'area_mean', 'smoothness_mean','concavity_mean', 'symmetry_mean']].describe())

In [ ]:
       perimeter_mean   area_mean  ...  concavity_mean  symmetry_mean
count       73.000000   73.000000  ...       73.000000      73.000000
mean        69.395068  359.480822  ...        0.041994       0.178968
std          3.148320   30.697175  ...        0.041429       0.027477
min         59.820000  278.600000  ...        0.000000       0.106000
25%         66.850000  334.200000  ...        0.015480       0.163300
50%         70.310000  365.600000  ...        0.030460       0.177900
75%         71.760000  386.300000  ...        0.053970       0.192700
max         74.650000  404.900000  ...        0.228000       0.274300

#### Conclusiones del Ejercicio

En este caso, la relación entre el área media y el perímetro medio es aproximadamente de 4.89, pero el área media tiene una desviación estándar más grande (de hecho, su valor máximo es de alrededor de 410). La concavidad media es extremadamente pequeña en comparación con la anterior (incluso con aproximadamente la misma desviación estándar), mientras que la simetría media es casi equivalente. A partir de este breve análisis, podemos deducir que la simetría media no es una característica discriminante, mientras que una relación entre el área media y el perímetro medio menor que 5.42 (considerando los valores máximos) con una concavidad media menor o igual a 0.04 debería garantizar un resultado benigno. Dado que la concavidad media puede alcanzar un valor máximo muy grande (mayor que el asociado con muestras malignas), es necesario considerar también las otras características para decidir si su valor debe ser considerado como una alarma. Sin embargo, podemos concluir diciendo que todas las muestras pertenecientes a estos grupos son benignas con una probabilidad de error insignificante

Es importante recalcar que esto es más un ejercicio que un análisis real y, en tales situaciones, la tarea principal del científico de datos es recopilar piezas de información contextual que puedan respaldar las conclusiones. Incluso en presencia de la verdad fundamental, este proceso de validación es siempre obligatorio porque la complejidad de las causas subyacentes puede llevar a afirmaciones y reglas completamente erróneas

### Coeficiente de Silhouette
El método más común para evaluar el rendimiento de un algoritmo de agrupamiento sin conocimiento de la verdad fundamental es el puntaje de silueta. Proporciona tanto un índice por muestra como una representación gráfica global que muestra el nivel de coherencia interna y separación de los grupos. Para calcular el puntaje, necesitamos introducir dos medidas auxiliares. La primera es la distancia intra-cluster promedio de una muestra $x_i ∈ K_j$ asumiendo la cardinalidad de $|K_j|=n(j)$:

$$ a(x_i)=\frac{1}{n(j)}\sum_{x_t \in K_j} d(x_i,x_t) $$

Por supuesto, $d(•)$ debe ser la misma función de distancia empleada en el procedimiento de agrupamiento. Dada una muestra $x_i ∈ K_j$, vamos a denotar el grupo más cercano como $K_c$. De esta manera, también podemos definir la distancia más pequeña al grupo más cercano (como la distancia promedio al grupo más cercano)

$$ b(x_i)=\frac{1}{n(c)}\sum_{x_t \in K_c} d(x_i,x_t) $$

Con estas dos medidas, podemos definir el coeficiente de Silhouette para cada $x_i \in X$

$$ s(x_i)=\frac{b(x_i)-a(x_i)}{\max\{a(x_i),b(x_i)\}}$$

El puntaje $s(•) ∈ (-1, 1)$. Cuando $s(•) → -1$, significa que $b(•) << a(•)$, por lo tanto, la muestra $x_i ∈ K_j$ está más cerca del grupo más cercano $K_c$ que de las otras muestras asignadas a $K_j$. Esta condición indica una asignación incorrecta. Por otro lado, cuando $s(•) → 1$, $b(•) >> a(•)$, por lo que la muestra $x_i$ está mucho más cerca de sus vecinos (pertenecientes al mismo grupo) que de cualquier otro punto asignado al grupo más cercano. Claramente, esta es una condición óptima y la referencia a emplear al ajustar un algoritmo. Sin embargo, como este índice no es global, es útil introducir gráficos de silueta, que muestran los puntajes obtenidos por cada muestra, agrupados por grupo y ordenados de manera descendente. Consideremos gráficos de Silhouette para el conjunto de datos de cancer de pecho para $K={2, 4, 6, 8}$

<img src="https://drive.google.com/uc?export=view&id=1wbGpgstZnxNs6k9_HI9UPjqNm0NBdrDN" width = "800" align="center"/>

In [ ]:
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm
nb_clusters = [2, 4, 6, 8]
mapping = [(0, 0), (0, 1), (1, 0), (1, 1)]

nb_clusters = [2, 4, 6, 8]
mapping = [(0, 0), (0, 1), (1, 0), (1, 1)]

# Show the silhouette plots
fig, ax = plt.subplots(2, 2, figsize=(15, 10))
for i, n in enumerate(nb_clusters):
    km = KMeans(n_clusters=n, random_state=1000)
    Y = km.fit_predict(cdf)
    df_km = pd.DataFrame(Y_pred, columns=['prediction'], index=cdf.index)
    kmdff = pd.concat([dff, df_km], axis=1)
    silhouette_values = silhouette_samples(cdf, kmdff['prediction'])
    ax[mapping[i]].set_xticks([-0.15, 0.0, 0.25, 0.5, 0.75, 1.0])
    ax[mapping[i]].set_yticks([])
    ax[mapping[i]].set_title('%d clusters' % n)
    ax[mapping[i]].set_xlim([-0.15, 1])
    y_lower = 20
    for t in range(n):
        ct_values = silhouette_values[Y == t]
        ct_values.sort()
        y_upper = y_lower + ct_values.shape[0]
        color = cm.Accent(float(t) / n)
        ax[mapping[i]].fill_betweenx(np.arange(y_lower, y_upper), 0, ct_values, facecolor=color, edgecolor=color)
        y_lower = y_upper + 20
plt.show()

El primer diagrama muestra el agrupamiento natural con K=2. La primera silueta es muy marcada, lo que indica que la distancia inter-grupo promedio tiene una gran variación. Además, un grupo tiene muchas más asignaciones que el otro (aunque es menos marcado). A partir de la descripción del conjunto de datos, sabemos que las dos clases están desequilibradas (357 benignas frente a 212 malignas), por lo tanto, la asimetría está parcialmente justificada. Sin embargo, en general, cuando los conjuntos de datos están equilibrados, un buen gráfico de silueta se caracteriza por tener grupos homogéneos con una silueta redondeada que debería estar cerca de 1.0. De hecho, cuando la forma es similar a un cigarro largo, significa que las distancias intra-grupo están muy cerca de su promedio (alta cohesión) y hay una clara separación entre los grupos adyacentes. Para K=2, tenemos puntajes razonables, ya que el primer grupo alcanza 0.6, mientras que el segundo tiene un pico correspondiente a alrededor de 0.8. Sin embargo, aunque en este último la mayoría de las muestras se caracterizan por tener s(•) > 0.75, en el primero, aproximadamente la mitad de las muestras están por debajo de 0.5. Este análisis muestra que el grupo más grande es más homogéneo y es más fácil para K-medias asignar las muestras (es decir, en términos de medidas, la varianza de xi ∈ K2 es menor y, en el espacio de alta dimensión, la esfera que representa a K2 es más uniforme que la que representa a K1)

Los demás gráficos muestran escenarios similares porque se ha detectado un grupo muy cohesivo junto con algunos grupos agudos. Esto significa que hay una discrepancia de anchura muy consistente. Sin embargo, al aumentar K, obtenemos grupos ligeramente más homogéneos porque tiende a ser similar el número de muestras asignadas. La presencia de un grupo muy redondeado (casi rectangular) con s(•) > 0.75 confirma que el conjunto de datos contiene al menos un grupo de muestras muy cohesivas, cuyas distancias con respecto a cualquier otro punto asignado a otros grupos son bastante cercanas. Sabemos que la clase maligna (aunque su cardinalidad sea mayor) es más compacta, mientras que la benigna se extiende sobre un subespacio mucho más amplio; por lo tanto, podemos asumir que para todos los K, el grupo más redondeado está compuesto por muestras malignas y todos los demás se pueden distinguir según su nitidez. Por ejemplo, para K=8, es muy probable que el tercer grupo corresponda a la parte central del segundo grupo en el primer gráfico, mientras que los más pequeños contienen muestras que pertenecen a regiones aisladas del subconjunto benigno.

Si no conocemos la verdad fundamental, deberíamos considerar tanto K=2 como K=8 (o incluso un número mayor). De hecho, en el primer caso, probablemente estemos perdiendo muchos detalles finos de información, pero estamos determinando una subdivisión sólida (asumiendo que un grupo no es extremadamente cohesivo debido a la naturaleza del problema). Por otro lado, con K>8, los grupos son obviamente más pequeños, con una cohesión moderadamente mayor, y representan subgrupos que comparten algunas características comunes. Como discutimos en la sección anterior, la elección final depende de muchos factores y estas herramientas solo pueden proporcionar una indicación general. Además, cuando los grupos no son convexos o su varianza no está distribuida uniformemente entre todas las características, K-medias siempre dará rendimientos subóptimos porque los grupos resultantes incorporarán un espacio grande vacío. Sin direcciones específicas, el número óptimo de grupos está asociado con un gráfico que contiene gráficos redondeados homogéneos (con aproximadamente el mismo ancho). Si la forma permanece afilada para cualquier valor de K, significa que la geometría no es completamente compatible con medidas simétricas (por ejemplo, los grupos están muy estirados) y se deben considerar otros métodos.


#### Índice de Completitud
Esta medida (junto con todas las demás discutidas a partir de ahora) se basa en el conocimiento de la verdad fundamental. Antes de introducir el índice, es útil definir algunos valores comunes. Si denotamos con $Y_{true}$ al conjunto que contiene las asignaciones verdaderas y con $Y_{pred}$ al conjunto de predicciones (ambos conteniendo $M$ valores y $K$ grupos), podemos estimar las siguientes probabilidades. $n_{true}(k)$ y $n_{pred}(k)$ representa el número de muestras verdaderas/predichas que pertenecen al grupo $k ∈ K$. En este punto, podemos calcular las entropías de $Y_{true}$ y $Y_{pred}$.

$p(y_{true}=k \in K)=\frac{n_{true}(k)}{M}$

$p(y_{pred}=k \in K)=\frac{n_{pred}(k)}{M}$

$H(Y_{true})=-\sum_{k}\frac{n_{true}(k)}{M}\log(\frac{n_{true}(k)}{M})$

$H(Y_{pred})=-\sum_{k}\frac{n_{pred}(k)}{M}\log(\frac{n_{pred}(k)}{M})$

Considerando la definición de entropía, $H(•)$ se maximiza con una distribución uniforme, la cual, a su vez, corresponde a la máxima incertidumbre de cada asignación. Para nuestros propósitos, también es necesario introducir las entropías condicionales (que representan la incertidumbre de una distribución dada el conocimiento de otra) de $Y_{true}$ dado $Y_{pred}$ y viceversa.

$H(Y_{true}|Y_{pred})=-\sum_{i}\sum_{j}\frac{n(i,j)}{M}\log(\frac{n(i,j)}{n_{pred}(j)})$

$H(Y_{pred}|Y_{true})=-\sum_{i}\sum_{j}\frac{n(i,j)}{M}\log(\frac{n(i,j)}{n_{true}(i)})$

Donde la función $n(i,j)$ representa, en la primera ecuación, el número de muestras con la etiqueta verdadera de $i$ asignadas al grupo $K_j$. En la segunda ecuación, $n(i,j)$ representa el número de muestras con la etiqueta verdadera de $j$ asignadas al grupo $K_i$.

Finalmente, el índice de completitud se define como.

$$c=1-\frac{H(Y_{pred}|Y_{true})}{H(Y_{true})}$$

Cuando $H(Y_{pred}|Y_{true}) → 0$, el conocimiento de $Y_{true}$ reduce la incertidumbre de las predicciones y, por lo tanto, c → 1. Esto es equivalente a decir que todas las muestras con la misma etiqueta verdadera se asignan al mismo grupo. Por el contrario, cuando $H(Y_{pred}|Y_{true} → H(Y_{pred}$, significa que la verdad fundamental no proporciona ninguna información que reduzca la incertidumbre de las predicciones y $c → 0$.

Por supuesto, un buen agrupamiento se caracteriza por $c → 1$. En el caso del conjunto de datos de cáncer de mama de Wisconsin, el puntaje de completitud, calculado utilizando la función ```completenss_score()``` de ```scikit-learn``` (que también funciona con etiquetas de texto) y $K=2$ (la única configuración asociada con la verdad fundamental), es de 0.517. Este resultado confirma que, para $K=2$, K-medias no puede separar perfectamente los grupos, porque, como hemos visto, hay algunas muestras malignas que se asignan erróneamente al grupo que contiene la gran mayoría de las muestras benignas. Sin embargo, como $c$ no es extremadamente pequeña, podemos estar seguros de que la mayoría de las muestras para ambas clases han sido asignadas a diferentes grupos. Como se menciona anteriormente, dada la geometría de los datos es recomendable explorar otros métodos de agrupación.

#### Índice de Homogeneidad
El indice de homogeneidad es complementario al anterior y se basa en la suposición de que un grupo debe contener solo muestras que tengan la misma etiqueta verdadera. Se define como:

$$c=1-\frac{H(Y_{true}|Y_{pred})}{H(Y_{true})}$$

Similarmente al indice de completitud, cuando $H(Y_{true}|Y_{pred} → H(Y_{true}$, significa que las asignaciones no tienen impacto en la entropía condicional, por lo tanto, la incertidumbre no se reduce después del agrupamiento (por ejemplo, cada grupo contiene muestras pertenecientes a todas las clases) y h → 0. Por el contrario, cuando H(Ytrue|Ypred) → 0, h → 1, porque el conocimiento de las predicciones ha reducido la incertidumbre sobre las asignaciones verdaderas y los grupos contienen casi exclusivamente muestras con la misma etiqueta. Es importante recordar que este indice por sí solo no es suficiente, porque no garantiza que un grupo contenga todas las muestras xi ∈ X con la misma etiqueta verdadera. Por eso, el indice de homogeneidad siempre se evalúa junto con el indice de completitud. 

El resultado del test es de 0.422. Este valor (especialmente para $K=2$) confirma nuestro análisis inicial. Al menos un grupo (el que contiene la mayoría de las muestras benignas) no es completamente homogéneo, ya que contiene muestras pertenecientes a ambas clases. Sin embargo, como el valor no está muy cerca de 0, podemos estar seguros de que las asignaciones son parcialmente correctas. Considerando ambos valores, h y c, podemos deducir que K-medias no está funcionando extremadamente bien (probablemente debido a la no convexidad), pero al mismo tiempo, es capaz de separar correctamente todas esas muestras cuya distancia al grupo más cercano está por encima de un umbral específico.

Es evidente que, con conocimiento de la verdad fundamental, no podemos aceptar fácilmente los resultados de K-medias y deberíamos buscar otro algoritmo que pueda proporcionar tanto $h$ como $c → 1$, es decir, que logre una alta homogeneidad y completitud en la asignación de las muestras a los grupos.

#### Un Equilibrio entre Homogeneidad y Completitud utilizando la Medida V

El lector familiarizado con el aprendizaje supervisado debería conocer el concepto de puntuación F (o medida F), que es la media armónica de precisión y recall. El mismo tipo de compensación puede ser empleado también al evaluar resultados de agrupamiento dados la verdad fundamental. De hecho, en muchos casos, es útil tener una única medida que tenga en cuenta tanto la homogeneidad como la completitud. Tal resultado se puede lograr fácilmente utilizando la medida V (o puntuación V), que se define como.

$$V = \frac{2}{\frac{1}{Homogeneidad}+\frac{1}{Completitud}} = \frac{2\cdot Homogeneidad \cdot Completitud}{Homogeneidad+Completitud}$$

Para nuestro caso, el valor del índice es de 0.465. Como se esperaba, la puntuación V es una medida promedio que, en este caso, se ve negativamente influenciada por una homogeneidad más baja. Por supuesto, este índice no proporciona ninguna información diferente, por lo tanto, solo es útil para sintetizar la completitud y la homogeneidad en un único valor. Sin embargo, con algunas manipulaciones matemáticas simples pero tediosas, es posible demostrar que la medida V también es simétrica (es decir, V(Ypred|Ytrue) = V(Ytrue|Ypred)); por lo tanto, dadas dos asignaciones independientes Y1 e Y2, V(Y1|Y2) es una medida de acuerdo entre ellas. Tal escenario no es extremadamente común, porque otras medidas pueden lograr un mejor resultado. Sin embargo, este puntaje podría ser utilizado, por ejemplo, para verificar si dos algoritmos (posiblemente basados en estrategias diferentes) tienden a producir las mismas asignaciones o si son discordantes. En este último caso, incluso si la verdad fundamental es desconocida, el científico de datos puede comprender que una estrategia seguramente no es tan efectiva como la otra y comenzar un proceso de exploración para encontrar el algoritmo de agrupamiento óptimo.



In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_mutual_info_score, adjusted_rand_score
# Compute the other metrics for K=2
km = KMeans(n_clusters=2, max_iter=1000, random_state=1000)
Y_pred = km.fit_predict(cdf)
df_km = pd.DataFrame(Y_pred, columns=['prediction'], index=cdf.index)
kmdff = pd.concat([dff, df_km], axis=1)

print('Completeness: {}'.format(completeness_score(kmdff['diagnosis'], kmdff['prediction'])))
print('Homogeneity: {}'.format(homogeneity_score(kmdff['diagnosis'], kmdff['prediction'])))
print('V-Score: {}'.format(v_measure_score(kmdff['diagnosis'], kmdff['prediction'])))
print('Adj. Mutual info: {}'.format(adjusted_mutual_info_score(kmdff['diagnosis'], kmdff['prediction'])))
print('Adj. Rand score: {}'.format(adjusted_rand_score(kmdff['diagnosis'], kmdff['prediction'])))